In [1]:
import pandas as pd 
from data_transform import wide_date_conv

pd.set_option('display.max_columns', 500)

# Proof of Concept

In [2]:
df = pd.DataFrame({2016: [1,2], "2018-4-3": [3,4], 'blah': [5,6], 2018: [7,8], 'blarg': [7,8]})
df.columns

Index([2016, '2018-4-3', 'blah', 2018, 'blarg'], dtype='object')

In [3]:
# force column headers to datetime
numeric_col_df = df.copy()
numeric_col_df.columns = pd.to_datetime(numeric_col_df.columns, errors = 'coerce')
numeric_col_df.head()

,1970-01-01 00:00:00.000002016,2018-04-03 00:00:00.000000000,NaT,1970-01-01 00:00:00.000002018,NaT
0,1,3,5,7,7
1,2,4,6,8,8


In [4]:
# idenify column header values to keep
columns_to_keep = numeric_col_df.columns.dropna()
columns_to_keep

DatetimeIndex(['1970-01-01 00:00:00.000002016',
                         '2018-04-03 00:00:00',
               '1970-01-01 00:00:00.000002018'],
              dtype='datetime64[ns]', freq=None)

In [5]:
# get index values for column headers that are numeric
date_cols_idx = numeric_col_df.columns.get_indexer_for(columns_to_keep)
date_cols_idx

array([0, 1, 3], dtype=int64)

In [6]:
# select only the columns that have date-like formats by found indexes
date_cols = df.iloc[:,date_cols_idx].columns

# select non date columns
non_date_cols = df.columns[~df.columns.isin(date_cols)]

In [7]:
long_df = pd.melt(df, id_vars=non_date_cols, var_name="period", value_name="value")
long_df

,blah,blarg,period,value
0,5,7,2016,1
1,6,8,2016,2
2,5,7,2018-4-3,3
3,6,8,2018-4-3,4
4,5,7,2018,7
5,6,8,2018,8


# Actual Examples

## Example 1: 
no column to widen and create new discrete columns

In [8]:
# import
dept_df = pd.read_excel("data/Total Outlays by Dept.xlsx", skiprows=2)

# pre transformation cleaning
dept_df = dept_df.drop(columns=['TQ'])
dept_df.columns = dept_df.columns.str.replace(" estimate", "")
dept_df.head()

,Department or other unit,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,Legislative Branch,196,192,199,212,234,252,269,289,353,395,499,553,638,739,788,990,1064,1099,1224,1214,1367,1437,1577,1607,1662,1810,1848,2216,2231,2245,2629,2394,2537,2609,2263,2348,2572,2592,2871,2959,3187,3396,3890,3984,4101,4294,4408,4704,5839,4582,4440,4316,4164,4321,4347,4499,4675,4961,6048,5795,5815,5765,5752,5746
1,Judicial Branch,57,62,66,75,80,88,94,110,133,145,173,188,207,284,325,393,437,481,567,641,710,787,866,966,1071,1180,1342,1499,1646,1997,2308,2628,2677,2903,3059,3259,3459,3789,4057,4381,4828,5127,5389,5547,5823,6006,6347,6645,7181,7293,7227,7063,6893,7137,7499,7569,7778,7988,8580,8993,8980,8940,8966,8989
2,Department of Agriculture,6437,7414,7569,6940,5633,5952,7430,8446,8412,8673,11021,10167,10302,15518,17682,23287,30179,31698,34721,41541,45623,52317,41928,55435,58599,49507,43930,48256,45858,53990,56320,63019,60615,56550,54218,52393,53800,62690,75071,68071,68622,72737,71560,85308,93533,84427,90795,114440,129459,139397,139717,155895,141808,139115,138163,127560,136716,150125,154617,129215,125209,124623,121982,122691
3,Department of Commerce,215,354,702,736,485,477,582,607,778,783,850,934,992,1077,1484,2010,4720,3538,3129,2296,2054,1925,1894,2139,2083,2127,2278,2570,3734,2585,2566,2797,2915,3401,3702,3782,4036,5020,7788,5003,5312,5665,5829,6147,6372,6475,7721,10718,13236,9930,10267,9140,7895,8956,9165,10310,8565,11323,17124,12245,12269,8814,8380,8151
4,Department of Defense--Military Programs,50111,51147,52585,48780,56629,70069,80355,80771,80123,77497,77645,75033,77864,84852,87917,95147,102259,113605,130912,153861,180693,204356,220863,245109,265440,273919,281889,294829,289694,261860,286574,278510,268577,259487,253196,258262,255793,261196,281028,290185,331845,388686,437034,474354,499344,528578,594662,636775,666715,678074,650867,607800,577898,562499,565370,568896,600683,653992,689589,729329,744099,745423,751666,762194


In [9]:
# using the transformation function
long_dept_df = wide_date_conv(dept_df)
long_dept_df.head(10)

,Department or other unit,period,value
0,Legislative Branch,1962,196
1,Judicial Branch,1962,57
2,Department of Agriculture,1962,6437
3,Department of Commerce,1962,215
4,Department of Defense--Military Programs,1962,50111
5,Department of Education,1962,816
6,Department of Energy,1962,2755
7,Department of Health and Human Services,1962,3529
8,Department of Homeland Security,1962,566
9,Department of Housing and Urban Development,1962,826


## Example 2
column to widen (pivot) into discrete columns

In [10]:
# import
taxes_df = pd.read_csv("data/Personal Income and Taxes by State.csv", skiprows=4, skipfooter=9, engine="python")

# drop NaN
taxes_df.dropna(subset=["GeoName", "LineCode"], inplace=True)
taxes_df.drop(columns=["LineCode"], inplace=True)

taxes_df.head()

,GeoFips,GeoName,Description,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,0,United States *,Personal income (thousands of dollars),1.495704e+09,1.651632e+09,1.855849e+09,2.073257e+09,2.313160e+09,2.592915e+09,2.779794e+09,2.968676e+09,3.279488e+09,3.510471e+09,3.719647e+09,3.946593e+09,4.267813e+09,4.609667e+09,4.897821e+09,5.067291e+09,5.409920e+09,5.648732e+09,5.940128e+09,6.286143e+09,6.673186e+09,7.086935e+09,7.601594e+09,8.001563e+09,8.650325e+09,9.001839e+09,9.155663e+09,9.480901e+09,1.002878e+10,1.059395e+10,1.137259e+10,1.200220e+10,1.243853e+10,1.205131e+10,1.254200e+10,1.331548e+10,1.399838e+10,1.417550e+10,1.498272e+10,1.571714e+10,1.615188e+10,1.693758e+10,1.783926e+10,1.854226e+10
1,0,United States *,Less: Personal current taxes,1.722160e+08,1.973640e+08,2.290960e+08,2.682560e+08,2.987640e+08,3.448810e+08,3.537710e+08,3.519590e+08,3.769730e+08,4.167680e+08,4.367350e+08,4.884010e+08,5.047800e+08,5.664880e+08,5.933090e+08,5.874660e+08,6.117440e+08,6.477960e+08,6.922010e+08,7.475700e+08,8.362000e+08,9.308820e+08,1.031430e+09,1.110884e+09,1.235250e+09,1.237927e+09,1.051183e+09,1.002446e+09,1.047624e+09,1.211185e+09,1.355468e+09,1.490831e+09,1.505811e+09,1.150907e+09,1.236004e+09,1.451655e+09,1.507470e+09,1.674228e+09,1.782338e+09,1.937443e+09,1.955616e+09,2.043573e+09,2.082114e+09,2.199985e+09
2,0,United States *,Equals: Disposable personal income,1.323488e+09,1.454268e+09,1.626753e+09,1.805001e+09,2.014396e+09,2.248034e+09,2.426023e+09,2.616717e+09,2.902515e+09,3.093703e+09,3.282912e+09,3.458192e+09,3.763033e+09,4.043179e+09,4.304512e+09,4.479825e+09,4.798176e+09,5.000936e+09,5.247927e+09,5.538573e+09,5.836986e+09,6.156053e+09,6.570164e+09,6.890679e+09,7.415075e+09,7.763912e+09,8.104480e+09,8.478455e+09,8.981157e+09,9.382761e+09,1.001712e+10,1.051137e+10,1.093272e+10,1.090040e+10,1.130599e+10,1.186382e+10,1.249091e+10,1.250128e+10,1.320038e+10,1.377970e+10,1.419626e+10,1.489401e+10,1.575714e+10,1.634228e+10
3,0,United States *,Population (persons) 1/,2.175539e+08,2.197609e+08,2.220982e+08,2.245686e+08,2.272247e+08,2.294657e+08,2.316644e+08,2.337920e+08,2.358249e+08,2.379237e+08,2.401328e+08,2.422889e+08,2.444990e+08,2.468192e+08,2.496228e+08,2.529809e+08,2.565142e+08,2.599186e+08,2.631258e+08,2.662784e+08,2.693943e+08,2.726469e+08,2.758541e+08,2.790402e+08,2.821624e+08,2.849690e+08,2.876252e+08,2.901079e+08,2.928053e+08,2.955166e+08,2.983799e+08,3.012312e+08,3.040940e+08,3.067715e+08,3.093217e+08,3.115569e+08,3.138310e+08,3.159937e+08,3.183010e+08,3.206352e+08,3.229413e+08,3.249855e+08,3.266875e+08,3.282395e+08
4,0,United States *,Per capita personal income (dollars) 2/,6.875000e+03,7.516000e+03,8.356000e+03,9.232000e+03,1.018000e+04,1.130000e+04,1.199900e+04,1.269800e+04,1.390600e+04,1.475500e+04,1.549000e+04,1.628900e+04,1.745500e+04,1.867600e+04,1.962100e+04,2.003000e+04,2.109000e+04,2.173300e+04,2.257500e+04,2.360700e+04,2.477100e+04,2.599300e+04,2.755700e+04,2.867500e+04,3.065700e+04,3.158900e+04,3.183200e+04,3.268100e+04,3.425100e+04,3.584900e+04,3.811400e+04,3.984400e+04,4.090400e+04,3.928400e+04,4.054700e+04,4.273900e+04,4.460500e+04,4.486000e+04,4.707100e+04,4.901900e+04,5.001500e+04,5.211800e+04,5.460600e+04,5.649000e+04


In [11]:
taxes_df_long = wide_date_conv(taxes_df)
taxes_df_long

,GeoFips,GeoName,Description,period,value
0,0,United States *,Personal income (thousands of dollars),1976,1.495704e+09
1,0,United States *,Less: Personal current taxes,1976,1.722160e+08
2,0,United States *,Equals: Disposable personal income,1976,1.323488e+09
3,0,United States *,Population (persons) 1/,1976,2.175539e+08
4,0,United States *,Per capita personal income (dollars) 2/,1976,6.875000e+03
...,...,...,...,...,...
18299,56000,Wyoming,Population (persons) 1/,2019,5.787590e+05
18300,56000,Wyoming,Per capita personal income (dollars) 2/,2019,6.218900e+04
18301,56000,Wyoming,Per capita disposable personal income (dollars...,2019,5.598900e+04
18302,56000,Wyoming,Personal current taxes,2019,3.588582e+06


In [14]:
taxes_df_long = wide_date_conv(taxes_df, repivot_col="Description")
taxes_df_long

,GeoFips,GeoName,period,Federal government 4/,Less: Personal current taxes,Equals: Disposable personal income,Per capita disposable personal income (dollars) 3/,Per capita personal income (dollars) 2/,Personal current taxes,Personal income (thousands of dollars),Population (persons) 1/
0,0,United States *,1976,141142000.0,172216000.0,1.323488e+09,6083.0,6875.0,172216000.0,1.495704e+09,217553859.0
1,0,United States *,1977,161998000.0,197364000.0,1.454268e+09,6618.0,7516.0,197364000.0,1.651632e+09,219760875.0
2,0,United States *,1978,188646000.0,229096000.0,1.626753e+09,7324.0,8356.0,229096000.0,1.855849e+09,222098244.0
3,0,United States *,1979,224224000.0,268256000.0,1.805001e+09,8038.0,9232.0,268256000.0,2.073257e+09,224568579.0
4,0,United States *,1980,249873000.0,298764000.0,2.014396e+09,8865.0,10180.0,298764000.0,2.313160e+09,227224719.0
...,...,...,...,...,...,...,...,...,...,...,...
2283,56000,Wyoming,2015,3388737.0,3522556.0,2.975204e+07,50805.0,56820.0,3522556.0,3.327459e+07,585613.0
2284,56000,Wyoming,2016,2964053.0,3099415.0,2.854629e+07,48863.0,54168.0,3099415.0,3.164571e+07,584215.0
2285,56000,Wyoming,2017,3185083.0,3326925.0,2.939672e+07,50778.0,56524.0,3326925.0,3.272364e+07,578931.0
2286,56000,Wyoming,2018,3219185.0,3378257.0,3.167597e+07,54841.0,60689.0,3378257.0,3.505423e+07,577601.0
